In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_3a_NonPath_SP1_ZyMo_PSL_1_2_179.csv') 

# split data
X=data.iloc[:,51:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

     Schneider.Xr.A  Schneider.Xr.R  ...  Grantham.Xd.29  Grantham.Xd.30
0          0.084182        0.061733  ...        0.032505        0.032749
1          0.120118        0.083347  ...        0.031626        0.032946
2          0.129916        0.043305  ...        0.031596        0.031098
3          0.124517        0.045875  ...        0.031188        0.031233
4          0.144744        0.057897  ...        0.031939        0.030060
..              ...             ...  ...             ...             ...
174        0.054376        0.088360  ...        0.029667        0.028973
175        0.067771        0.072984  ...        0.030044        0.026204
176        0.168879        0.038237  ...        0.031815        0.029722
177        0.087177        0.071793  ...        0.032362        0.030237
178        0.103912        0.055667  ...        0.032141        0.032526

[179 rows x 100 columns]
0      1
1      1
2      1
3      1
4      1
      ..
174    2
175    2
176    2
177    2
178    2

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(100, 2 - 1) = 1 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 207,586
Trainable params: 207,586
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
5/5 [==============================] - 37s 722ms/step - loss: 0.6916 - accuracy: 0.5828 - val_loss: 0.6852 - val_accuracy: 1.0000
Epoch 2/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6832 - accuracy: 0.9265 - val_loss: 0.6744 - val_accuracy: 1.0000
Epoch 3/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.6715 - accuracy: 0.9466 - val_loss: 0.6582 - val_accuracy: 1.0000
Epoch 4/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.6528 - accuracy: 0.9397 - val_loss: 0.6337 - val_accuracy: 0.9444
Epoch 5/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6298 - accuracy: 0.9265 - val_loss: 0.5979 - val_accuracy: 0.9444
Epoch 6/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.5953 - accuracy: 0.9339 - val_loss: 0.5486 - val_accuracy: 0.9444
Epoch 7/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.5513 - accuracy: 0.9318 - val_loss: 0.4861 - val_accuracy: 0.9444
Epoch 8/100

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.958041958041958
test-acc = 0.9722222222222222
train-CFR =               precision    recall  f1-score   support

           0       0.90      0.82      0.86        22
           1       0.97      0.98      0.98       121

    accuracy                           0.96       143
   macro avg       0.93      0.90      0.92       143
weighted avg       0.96      0.96      0.96       143

test-CFR =               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.97      0.98        30

    accuracy                           0.97        36
   macro avg       0.93      0.98      0.95        36
weighted avg       0.98      0.97      0.97        36

